In [3]:
!pip install opencv-python matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.1/31.1 MB 11.0 MB/s eta 0:00:0000:0100:01


In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.9.0
Keras Version: 2.9.0

Python 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:29) [Clang 14.0.6 ]
Pandas 1.5.2
Scikit-Learn 1.2.0
GPU is available


In [5]:
!pip install labelme

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 5.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 6.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
      Traceback (most recent call last):
        File "/Users/gryffindorgirl/miniconda/envs/tensorflow/lib/python3.10/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 144, in pre

In [2]:
!pip install albumentations

In [2]:
import cv2
import os
import random
import numpy as np
import uuid
from matplotlib import pyplot as plt

## import tensorflow dependencies

In [3]:
from tensorflow.keras.models import Model

In [4]:
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

# Set GPU Growth

In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
for gpu in gpus:
    print(gpu)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


# Create Folder Structures

In [7]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [14]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# Dataset

In [15]:
#negatives-> from LFW = > Labelled faces in the WILD
for folder in os.listdir('lfw'):
    for img in os.listdir(os.path.join('lfw', folder)):
        org_path = os.path.join('lfw', folder, img)
        new_path = os.path.join(NEG_PATH, img)
        os.replace(org_path, new_path)

In [7]:
#collect positive images and anchor images
# cap = cv2.VideoCapture(0)
# while cap.isOpened():
#     ret, frame = cap.read()
    
#     frame = frame[120:120+250 , 200:200+250, :]
    
#     if cv2.waitKey(1) & 0XFF == ord('a'):
#         #capture anchor image
#         imgname = os.path.join(ANC_PATH , '{}.jpg'.format(uuid.uuid1()))
#         cv2.imwrite(imgname, frame)
    
#     if cv2.waitKey(1) & 0XFF == ord('p'):
#         #capture positive image
#         imgname = os.path.join(POS_PATH , '{}.jpg'.format(uuid.uuid()))
#         cv2.imwrite(imgname, frame)
        
#     cv2.imshow('Image Collection', frame)
    
#     if cv2.waitKey(1) & 0XFF & ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()


cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[400:400+250,500:500+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()


In [28]:
#data augmentation using albumentations
def data_aug(img):
    data = []
    for i in range(4):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        img = tf.image.stateless_random_hue(img, max_delta=0.2, seed=(1,2))
        img = tf.image.stateless_random_saturation(img, lower=0.9, upper=1, seed=(np.random.randint(100), np.random.randint(100)))
        
        data.append(img)
        
    return data 

In [29]:
for fileName in os.listdir(os.path.join(POS_PATH)):
    img = cv2.imread(os.path.join(POS_PATH , fileName))
    augmented_images = data_aug(img)
    for aug_img in augmented_images:
        cv2.imwrite(os.path.join(POS_PATH,'{}.jpg'.format(uuid.uuid1())) , aug_img.numpy())

KeyboardInterrupt: 

# Load and Preprocess the data

In [64]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(3000)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(3000)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(3000)

In [8]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (100,100))
    img = img/255.0  #normalisation
    return img

# Create Labelled Dataset

In [66]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [9]:
def preprocess_twin(input_image , validation_image, label):
    return (preprocess(input_image) , preprocess(validation_image) , label)

In [68]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=10000)

In [69]:
#Training partition
train_data = data.take(round(len(data)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [70]:
#Testing partition
test_data = data.skip(round(len(data)*0.7))
test_data = test_data.take(round(len(data)*0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# Model Engineering

## Building Embedding Model

In [ ]:
inp = Input(shape=(100,100,3) , name='input_image')

In [ ]:
c1 = Conv2D(64, (10,10) , activation = 'relu')(inp)

In [ ]:
m1 = MaxPooling2D(64, (2,2) , padding = 'same')(c1)

In [ ]:
c2 = Conv2D(128 , (7,7), activation='relu')(m1)
m2 = MaxPooling2D(64, (2,2) , padding='same')(c2)

In [ ]:
c3 = Conv2D(128, (4,4) , activation='relu')(m2)
m3 = MaxPooling2D(64, (2,2) , padding='same')(c3)

In [ ]:
c4 = Conv2D(256, (4,4) , activation='relu')(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation='sigmoid')(f1)

In [ ]:
mod = Model(inputs=[inp], outputs=[d1] , name='embedding')

In [10]:
def make_embedding():
    inp = Input(shape=(100,100,3) , name='input_image')
    
    c1 = Conv2D(64, (10,10) , activation = 'relu')(inp)
    m1 = MaxPooling2D(64, (2,2) , padding = 'same')(c1)
    
    c2 = Conv2D(128 , (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2) , padding='same')(c2)
    
    c3 = Conv2D(128, (4,4) , activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2) , padding='same')(c3)
    
    c4 = Conv2D(256, (4,4) , activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    mod = Model(inputs=[inp], outputs=[d1] , name='embedding')
    
    return mod

In [11]:
embedding = make_embedding()

Metal device set to: Apple M1


2023-01-22 23:33:24.125706: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-22 23:33:24.125976: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128)       26

## Build the Distance Layer

In [13]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

# Make the Siamese Model

In [14]:
def make_siamese_model():
    input_image = Input(name='input_img' , shape=(100,100,3))
    validation_image = Input(name='val_img' , shape=(100,100,3))
    
    siamese_layer = L1Dist()
    siamese_layer._name = 'Distance'
    distances  = siamese_layer(embedding(input_image), embedding(validation_image))
    
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image,validation_image] , outputs=classifier , name='SiameseNetwork')

In [15]:
siamese_model = make_siamese_model()

In [16]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 val_img (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'val_img[0][0]']   

# Training

## Setup Loss and Optimizer

In [17]:
binary_cross_entropy_loss = tf.keras.losses.BinaryCrossentropy()

In [18]:
opt = tf.keras.optimizers.Adam(1e-4)  #i.e; 0.0001

## Establish Checkpoints

In [24]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt = opt, siamese_model = siamese_model)

## Build Train Step Function

In [26]:
@tf.function
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_entropy_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
    # Return loss
    return loss

## Build the training Loop

In [19]:
from tensorflow.keras.metrics import Precision, Recall

In [20]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

## Train the Model

In [28]:
EPOCHS = 50

In [29]:
train(train_data, EPOCHS)


 Epoch 1/50


2023-01-21 14:25:28.517560: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)


2023-01-21 14:25:30.262846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


  1/263 [..............................] - ETA: 19:59

...

263/263 [==============================] - 726s 3s/step
1.1361568e-07 1.0 1.0

 Epoch 46/50
1/1 [==============================] - 3s 3s/step:


263/263 [==============================] - 766s 3s/step
0.033112627 0.96365374 0.98871446

 Epoch 47/50
1/1 [==============================] - 3s 3s/step:


263/263 [==============================] - 779s 3s/step
0.004168094 0.9943768 0.9920524

 Epoch 48/50
1/1 [==============================] - 2s 2s/step:


263/263 [==============================] - 723s 3s/step
0.0018438172 0.9971483 0.99384177

 Epoch 49/50
263/263 [==============================] - 969s 4s/step
0.020852577 0.99713606 0.9995215

 Epoch 50/50
1/1 [==============================] - 2s 2s/step0:


263/263 [==============================] - 644s 2s/step
0.00017563952 0.99905837 0.9981185


# Evaluate Model

In [100]:
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [101]:
yhat = siamese_model.predict([test_input,test_val])
yhat

1/1 [==============================] - 0s 19ms/step


array([[0.50150305],
       [0.5008036 ],
       [0.5062051 ],
       [0.5062437 ],
       [0.5036352 ],
       [0.50494176],
       [0.5054691 ],
       [0.5041795 ],
       [0.5008365 ],
       [0.5025595 ],
       [0.5017131 ],
       [0.5041885 ],
       [0.50193805],
       [0.5009412 ],
       [0.5012976 ],
       [0.50120896]], dtype=float32)

In [102]:
[1 if prediction > 0.5 else 0 for prediction in yhat]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [99]:
y_true


array([1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

# Save the model

In [35]:
siamese_model.save('siamesemodel.h5')

In [21]:
#reloading our model
model = tf.keras.models.load_model('siamesemodel.h5',
                                  custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [22]:
model.predict([test_input,test_val])

NameError: name 'test_input' is not defined

In [24]:
model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 val_img (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'val_img[0][0]']   

# Real Time Test

## Verification Function

In [23]:
def getNumberOfDirs():
    n = 0
    for folder in os.listdir(os.path.join('application_data','verification_images')):
        n+=1
    
    return n

In [24]:
def verify(model , detection_threshold , verification_threshold):
    n = getNumberOfDirs()
    names = []
    results = []
    maxVal=0
    for folder in os.listdir(os.path.join('application_data' , 'verification_images')):
#         names[].append(folder)
        resultsTemp = []
        for image in os.listdir(os.path.join('application_data','verification_images',folder)):
            input_image = preprocess(os.path.join('application_data' , 'input_image' , 'input_image.jpg'))
            validation_image = preprocess(os.path.join('application_data','verification_images',folder,image))
            
            result = model.predict(list(np.expand_dims([input_image,validation_image] , axis = 1)))
            resultsTemp.append(result)
        detection = np.sum(np.array(resultsTemp) > detection_threshold)    
        
        verification = detection/len(os.listdir(os.path.join('application_data','verification_images',folder)))
        verified = verification > verification_threshold
        
        if verified == True:
            return resultsTemp , verified , folder
    
    return [] , False , ""

## OpenCV Real Time Verification

In [25]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[400:400+250,500:500+250, :]
    cv2.imshow('Verification' , frame)
    
    if cv2.waitKey(10) & 0xFF == ord('v'):
        cv2.imwrite(os.path.join('application_data','input_image','input_image.jpg') , frame)
        results , verified , name = verify(model,0.7,0.5)
        print(verified,'\n',name)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 10ms/step


2023-01-22 23:34:26.553078: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-01-22 23:34:26.607675: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step
False 
 
1/1 [==============================] - 0s 9ms/step
True 
 Lakkhu
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step
True 
 Sunita
1/1 [==============================] - 0s 9ms/step


...

1/1 [==============================] - 0s 9ms/step
True 
 Garima
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step
True 
 Garima
1/1 [==============================] - 0s 10ms/step


1/1 [==============================] - 0s 10ms/step
False 
 
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step
False 
 
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 9ms/step
True 
 Garima


In [24]:
CREATE_PATH = os.path.join('application_data', 'verification_images' , 'Lakkhu')

In [25]:
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[400:400+250,500:500+250, :]
    
    # Collect anchors 
#     if cv2.waitKey(1) & 0XFF == ord('a'):
#         # Create the unique file path 
#         imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
#         # Write out anchor image
#         cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(CREATE_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()
